In [1]:
import requests, random, time
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
from IPython.display import clear_output

In [2]:
# Get base links
repos = [
    "html-css"
    "ethics",
    "python",
    "machine-learning",
    "text-analysis",
    "command-line",
    "twitter-api",
    "omeka",
    "databases",
    "project-lab",
    "git",
    "mapping"
]


to_visit, outlinks = [], []

for repo in repos:
    site = f"https://www.github.com/DHRI-Curriculum/{repo}"
    to_visit.append(site)

base = urlparse(site).netloc
visited, external_visited = {}, {}

s404 = []

In [3]:
previous = None
i = 1
while to_visit:
    l = to_visit.pop()

    if l.split(".")[-1].lower() in ['png', 'jpg', 'gif', 'ipynb', 'csv', 'jpeg']: continue # skip images and other formats
    
    clear_output(wait=True)
    print(f"{i}/{len(to_visit)}")
    i+=1
    
    '''
    _wait = random.randint(5,15)
    print(f"Waiting for {_wait}...")
    time.sleep(_wait)
    '''
    #print(l)
    url = urljoin(site, l)

    try:
        r = requests.get(url)
        visited[l] = r.status_code

    except:
        visited[l] = None

    if r.status_code == 429:
        raise RuntimeError("Reached 429...")
        
    if r.status_code == 404:
        print(f"404 found in {previous}: {r.url}")
        s404.append({'source': previous, 'target': r.url})
    
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html5lib')
        article = soup.find('article')
        try:
            links = [l['href'] for l in article.find_all('a', href=True)]
        except AttributeError:
            links = []
            print(f"An error occurred here - no 'article' element found in {r.url}.")
        for link in links:
            parsed_link = urlparse(link)
            loc = parsed_link.netloc
            path = parsed_link.path
            joined_url = urljoin(site, link)

            if loc == '':
                if joined_url not in to_visit and joined_url not in visited.keys():
                    to_visit.append(joined_url)

            elif loc == base:
                if link not in to_visit and link not in visited.keys():
                    to_visit.append(link)

            else:
                if link not in outlinks and link not in visited.keys():
                    outlinks.append(link)
    
    to_visit = list(set([x.split('#')[0] for x in to_visit])) # remove any anchors
    to_visit = [x for x in to_visit if x not in visited.keys()]
    
    previous = l
    
i = 1

# check the status of external links
while outlinks:
    l = outlinks.pop()
    
    clear_output(wait=True)
    print(f"{i}/{len(to_visit)}")
    i+=1

    try:
        r = requests.get(l)
        external_visited[l] = r.status_code

    except:
        external_visited[l] = None


225/0


In [4]:
import pandas as pd

s = pd.Series(visited, name='Response')
s.index.name = 'URL'
df1 = pd.DataFrame(s)
df1['Type'] = 'Internal'

s = pd.Series(external_visited, name='Response')
s.index.name = 'URL'
df2 = pd.DataFrame(s)
df2['Type'] = 'External'


In [5]:
df1.sort_values("Response")

,Response,Type
URL,,
https://www.github.com/DHRI-Curriculum/mapping,200,Internal
https://www.github.com/DHRI-Curriculum/mapping/blob/master/sections/15calc.md,200,Internal
https://www.github.com/DHRI-Curriculum/mapping/blob/master/sections/19layout.md,200,Internal
https://www.github.com/DHRI-Curriculum/python/blob/master/sections/06-loops.md,200,Internal
https://www.github.com/DHRI-Curriculum/python/blob/master/sections/solutions/loop.md,200,Internal
...,...,...
https://www.github.com/DHRI-Curriculum/command-line/blob/master/sections/11-summary.md,200,Internal
https://www.github.com/DHRI-Curriculum/omeka/blob/master/sections/theoreticalconcerns.md,200,Internal
https://www.github.com/DHRI-Curriculum/omeka/blob/master/sections/.../README.md,404,Internal


In [6]:
df2.sort_values("Response")

,Response,Type
URL,,
http://www.loc.gov/preservation/resources/rfs/TOC.html,200.0,External
http://www.todayifoundout.com/index.php/2015/04/c-drive-default-windows-based-computers-2/,200.0,External
https://genius.com/Johanna-drucker-from-a-to-screen-annotated,200.0,External
https://github.com/DHRI-Curriculum/glossary/blob/master/sections/command-line.md,200.0,External
https://youtu.be/Xq29uTtKW4M,200.0,External
...,...,...
https://de.dariah.eu/tatom/index.html,NaN,External
http://www.conf.org/whoweserve/community/resources/index.cfm?navitemNumber=15626#locator,NaN,External
http://dirtdirectory.org/,NaN,External


In [7]:
s404

[{'source': 'https://www.github.com/DHRI-Curriculum/mapping/blob/master/sections/4layerpr.md',
  'target': 'https://github.com/DHRI-Curriculum/html-cssethics'},
 {'source': 'https://www.github.com/DHRI-Curriculum/omeka/blob/master/sections/themesplugins.md',
  'target': 'https://github.com/DHRI-Curriculum/machine-learning/blob/master/sections/07-supervised.md'},
 {'source': 'https://www.github.com/DHRI-Curriculum/command-line/blob/master/sections/09-data.md',
  'target': 'https://github.com/DHRI-Curriculum/omeka/blob/master/sections/.../README.md'}]